<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from nltk.corpus import stopwords as nltk_stopwords, wordnet
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score
from IPython.display import display
from lightgbm import LGBMClassifier
from sklearn.utils import shuffle
RANDOM_STATE = 42
TEST_SIZE = 0.2

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv('D:/Yandex/toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
display(data.head())

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
display(data.tail())

,Unnamed: 0,text,toxic
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0
159291,159450,"""\nAnd ... I really don't think you understand...",0


In [6]:
display(data.sample(5))

,Unnamed: 0,text,toxic
33683,33723,"""—Preceding unsigned comment added by (talk •...",0
55400,55461,Did not know white men like Sitush also live i...,0
140790,140942,I don't understand. What is a vandalism blackl...,0
18139,18156,It seems Beve is more interested in having the...,0
1787,1787,"Nope, you're wrong. You can't block all possib...",0


In [7]:
data.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


In [8]:
data.duplicated().sum()

0

In [9]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [10]:
# Переводим буквы в нижний регистр, оставляем только латиницу, удаляет стоп-слова
def clear_text(text):
    stop_words = set(nltk_stopwords.words('english'))
    text = text.lower()
    word_list = re.sub(r"[^a-z ]", ' ', text).split()
    word_notstop_list = [w for w in word_list if not w in stop_words]
    return ' '.join(word_notstop_list)

In [11]:
# Добавим в датасет новый признак с очищенными комментариями
data['clean_text'] = data['text'].apply(clear_text)

In [12]:
display(data.sample(3))

,Unnamed: 0,text,toxic,clean_text
131482,131618,Wikipedia= Nazi Germany: Phase 2 \n\nHello. I ...,1,wikipedia nazi germany phase hello writing inf...
145464,145620,"""\n\n Yosting it \n\nAfter seven months on Wik...",0,yosting seven months wikipedia finally occurre...
43527,43578,The person who did the cut and paste move was ...,0,person cut paste move admin know trying help a...


# Лемматизируем комментарии
def lemm_text(text):
    lemmatizer = WordNetLemmatizer()
    word_list = text.split()
    lemmatized_text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])    
    return lemmatized_text

In [13]:
def wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN)

In [14]:
def lemm_text(text):
    lemmatizer = WordNetLemmatizer()
    word_list = text.split()
    lemmatized_text = ' '.join([lemmatizer.lemmatize(w, wordnet_pos(w)) for w in word_list])
    return lemmatized_text

In [15]:
data['cl_text'] = data['clean_text'].apply(lemm_text)

In [16]:
display(data.sample(3).T)

,153229,3637,102805
Unnamed: 0,153386,3637,102902
text,Fighting freedom-hating wikiterrorist/admins i...,November 2009 (UTC)\n\nThe context is evident ...,"To reiterate, please use edit summaries. Cheers!"
toxic,1,0,0
clean_text,fighting freedom hating wikiterrorist admins n...,november utc context evident movie understand ...,reiterate please use edit summaries cheers
cl_text,fight freedom hat wikiterrorist admins never w...,november utc context evident movie understand ...,reiterate please use edit summary cheer


- данные изучены
- комментарии очищены: буквы переведены в нижний регистр, оставлена только латиница, стоп-слова удалены
- комментарии лемматизированы
- пропусков и явных дубликатов в данных нет
- типы данных соответствуют требованиям для последующей очистки и лемматизации комментариев

## Обучение

In [17]:
kfold = KFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True)

In [18]:
features_train, features_test, target_train, target_test = train_test_split(data['cl_text'], data['toxic'].values, 
                                                                            test_size=TEST_SIZE, stratify=data['toxic'].values, 
                                                                            shuffle=True, random_state=RANDOM_STATE)

In [19]:
count_tf_idf = TfidfVectorizer()
tf_idf_train = count_tf_idf.fit_transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test)

In [20]:
# LogisticRegression
model_lr = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=RANDOM_STATE)
model_lr.f1 = cross_val_score(model_lr, tf_idf_train, target_train, cv=kfold, scoring='f1')
print('f1: %.3f' %(model_lr.f1.mean()))

f1: 0.752


In [21]:
# DecisionTreeClassifier
model_dt = DecisionTreeClassifier(max_depth=15, class_weight='balanced', random_state=RANDOM_STATE)
model_dt.f1 = cross_val_score(model_dt, tf_idf_train, target_train, cv=kfold, scoring='f1')
print('f1: %.3f' %(model_dt.f1.mean()))

f1: 0.613


In [22]:
# LGBMClassifier
model_lgbm = LGBMClassifier(n_estimators=300, max_depth=10, learning_rate=0.15, class_weight='balanced', 
                               boosting_type='gbdt', objective='binary', random_state=RANDOM_STATE)
model_lgbm.f1 = cross_val_score(model_lgbm, tf_idf_train, target_train, cv=kfold, scoring='f1')
print('f1: %.3f' %(model_lgbm.f1.mean()))

f1: 0.766


Выберем в качестве лучшей модель градиентного бустинга LGBMClassifier model_lgbm, которая имеет следующее значение метрики оценки качества на обучающей выборке:

- метрика f1: 0.766
- модель: LGBMClassifier
- количество деревьев: n_estimators = 300
- глубина дерева: max_depth = 10
- коэффициент скорости обучения: learning_rate = 0.15

## Выводы

In [23]:
# Тестирование лучшей модели
model_lgbm.fit(tf_idf_train, target_train)
model_lgbm.predicted = model_lgbm.predict(tf_idf_test)
model_lgbm.test_f1 = f1_score(target_test, model_lgbm.predicted)
print('f1: %.3f' %(model_lgbm.test_f1))

f1: 0.771


- данные изучены
- комментарии очищены: буквы переведены в нижний регистр, оставлена только латиница, стоп-слова удалены
- комментарии лемматизированы
- пропусков и явных дубликатов в данных нет
- типы данных соответствуют требованиям для последующей очистки и лемматизации комментариев
- данные разделены на обучающую и тестовую выборки, рассчитана величина TF-IDF, обучены три модели. 
- построили модель со значением метрики качества F1 выше 0.75.

Лучшая модель градиентного бустинга LGBMClassifier model_lgbm, которая имеет следующее значение метрики оценки качества:

- метрика f1 на обучающей выборке: 0.766
- метрика f1 на тестовой выборке: 0.771
- модель: LGBMClassifier
- количество деревьев: n_estimators = 300
- глубина дерева: max_depth = 10
- коэффициент скорости обучения: learning_rate = 0.15